## Gift Contract Testing

In [1]:
from web3.auto import w3
import pandas as pd
from cyberpy import generate_wallet, address_to_address
from cyberpy._message_signer import Message
from cyberpy._wallet import seed_to_privkey
from eth_account.messages import encode_defunct
import json
import random
from time import time
import ipfshttpclient
from tqdm.notebook import tqdm
from base64 import b64encode
from multiprocess import Pool
from math import ceil
from dotenv import dotenv_values

from contract_utils import execute_bash, instantiate_contract, execute_contract_bash, query_contract, get_ipfs_cid_from_str, get_proofs, ContractUtils


ipfs_client = ipfshttpclient.connect()

NUMBER_OF_PARTICIPANTS = 200_000
NUMBER_OF_ACTIVATED_PARTICIPANTS = 100_000
INITIAL_BALANCE = str(55_000_000_000)
COEF_UP = str(13)
COEF_DOWN = str(7)
TARGET_CLAIM = str(20_000)

NUMBER_OF_THREADS = 100
KEY_PHRASE = 'KEY PHRASE'
NICKNAME_LIST = [f'john{round(time())}{number}' for number in range(NUMBER_OF_PARTICIPANTS)]
AVATAR_CID_LIST = [get_ipfs_cid_from_str(_nickname + '_avatar') for _nickname in NICKNAME_LIST]
CLAIM_AMOUNT_LIST = [int(round(20_000 * random.random(), -3)) + 20_000 for _ in range(NUMBER_OF_PARTICIPANTS)]
PREFIXES = ['cosmos', 'osmo', 'terra']

WALLET_ADDRESS = dotenv_values('.env')['WALLET_ADDRESS']
WALLET_SEED = dotenv_values('.env')['WALLET_SEED']
PYTHON_PATH = dotenv_values('.env')['PYTHON_PATH']

DISPLAY_TX_EXECUTION = False
CALCULATE_ADDRESS_SET = True
CALCULATE_PROOFS = True
SEND_COINS = True

INIT_SUBGRAPH_CONTRACTS = False
SUBGRAPH_CODE_ID = str(3)
NAME_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom1qg5ega6dykkxc307y25pecuufrjkxkaggkkxh7nad0vhyhtuhw3sp98tza'
AVATAR_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom17p9rzwnnfxcjp32un9ug7yhhzgtkhvl9jfksztgw5uh69wac2pgs6qwudf'
PROOF_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom1unyuj8qnmygvzuex3dwmg9yzt9alhvyeat0uu0jedg2wj33efl5qpj0rgp'

INIT_PASSPORT_CONTRACT = False
PASSPORT_CODE_ID = str(2)
PASSPORT_CONTRACT_ADDRESS = 'bostrom1hulx7cgvpfcvg83wk5h96sedqgn72n026w6nl47uht554xhvj9nsjxcwgf'

INIT_TREASURY_CONTRACT = True
TREASURY_CODE_ID = str(4)
TREASURY_CONTRACT_ADDRESS = 'bostrom18cszlvm6pze0x9sz32qnjq4vtd45xehqs8dq7cwy8yhq35wfnn3q9u647v'

INIT_GIFT_CONTRACT = True
GIFT_CODE_ID = str(1)
GIFT_CONTRACT_ADDRESS = 'bostrom14swjuycwxmvghd9hgr3f3755388g6xkaqpn2dxcf700fzcwdm2nqum7453'

PARTICIPANTS_FILE_NAME = 'participants_test_data_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.csv'
ROOT_SOURCE_FILE_NAME = 'root_testing_source_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.json'
PROOF_FILE_NAME = 'proof_testing_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.json'
ROOT_FILE_NAME = 'root_testing_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses'

/home/grovybear/.local/lib/python3.9/site-packages/ipfshttpclient/client/__init__.py:75: VersionMismatch: Unsupported daemon version '0.12.2' (not in range: 0.5.0 ≤ … < 0.9.0)
  warnings.warn(exceptions.VersionMismatch(version, minimum, maximum))


### Generate addresses and sign messages

In [2]:
def generate_wallet_data(i: int, prefixes: list = PREFIXES) -> dict:

    prefix = random.choice(prefixes)

    claim_item = {'amount': CLAIM_AMOUNT_LIST[i],
              'nickname': NICKNAME_LIST[i],
              'avatar': AVATAR_CID_LIST[i]}

    # Generate Ethereum wallet
    ethereum_wallet = w3.eth.account.create(KEY_PHRASE)
    claim_item['ethereum_address'] = ethereum_wallet.address.lower()
    claim_item['ethereum_private_key'] = ethereum_wallet.privateKey.hex()

    # Generate Bostrom and Cosmos wallet
    bostrom_wallet = generate_wallet()
    claim_item['bostrom_address'] = bostrom_wallet['address']
    claim_item['cosmos_address'] = address_to_address(bostrom_wallet['address'], prefix)
    claim_item['cosmos_seed'] = bostrom_wallet['seed']

    # Create message
    claim_item['message'] = f"{claim_item['bostrom_address']}:QmRX8qYgeZoYM3M5zzQaWEpVFdpin6FvVXvp6RPQK3oufV"

    # Sign message form Ethereum address
    ethereum_signed_message = \
        w3.eth.account.sign_message(
            signable_message=encode_defunct(text=claim_item['message']),
            private_key=ethereum_wallet.privateKey)
    claim_item['ethereum_message_signature'] = ethereum_signed_message.signature.hex()

    # Sign message form Cosmos address
    cosmos_msg = Message(privkey=seed_to_privkey(seed=claim_item['cosmos_seed']))
    cosmos_msg.add_message(signing_message=claim_item['message'], signer_prefix=prefix)
    claim_item['cosmos_message_signed_row'] = cosmos_msg.get_signed_message()
    claim_item['cosmos_message_signature'] = b64encode(json.dumps(claim_item['cosmos_message_signed_row']).replace('\n', '').replace(' ', '').encode('utf-8')).decode("utf-8")

    # Verify message
    assert claim_item['ethereum_address'] == w3.eth.account.recover_message(
        signable_message=encode_defunct(text=claim_item['message']),
        signature=claim_item['ethereum_message_signature']).lower()

    return claim_item


if CALCULATE_ADDRESS_SET:

    tasks = list(range(NUMBER_OF_PARTICIPANTS))
    print(f'Number of tasks: {len(tasks)}')
    print(f'Number of threads: {NUMBER_OF_THREADS}')
    with Pool(processes=NUMBER_OF_THREADS) as pool:
        claims_list = list(tqdm(pool.imap(generate_wallet_data, tasks), total=len(tasks)))

    claims_df = pd.DataFrame(claims_list)
    claims_df.to_csv('claims_ethereum_test_data_without_proof_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.csv')
else:
    claims_df = pd.read_csv('claims_ethereum_test_data_without_proof_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.csv')
claims_df.head()

Number of tasks: 200000
Number of threads: 100


  0%|          | 0/200000 [00:00<?, ?it/s]

,amount,nickname,avatar,ethereum_address,ethereum_private_key,bostrom_address,cosmos_address,cosmos_seed,message,ethereum_message_signature,cosmos_message_signed_row,cosmos_message_signature
0,33000,john16517597490,Qme4FqA7kkwm4G14MEBXUGycmLN1j3CAaNgfYoysHZK5fF,0xb13349fc01611fb3d5eaa220ce20d7935503bfdb,0x87b3b8f2bb51d574cf3c97302f01bc71e0f77746d530...,bostrom16jeqp7f4s97g0rkq3wqdwwr34ntn2f503rj59z,terra16jeqp7f4s97g0rkq3wqdwwr34ntn2f5055u8e9,wheel saddle egg habit ribbon chase pilot focu...,bostrom16jeqp7f4s97g0rkq3wqdwwr34ntn2f503rj59z...,0x4feef418397ebfd0cae573e031bebe1f69d486c6ee50...,{'pub_key': 'AjX/LeG0AQo/SXxmETFE4msQBpGN1w028...,eyJwdWJfa2V5IjoiQWpYL0xlRzBBUW8vU1h4bUVURkU0bX...
1,36000,john16517597491,QmPwSc85Jmiw4mTS4esQNnPRwHgiXDFSVqPX7Dquv67L61,0xd4401c790797baff16eb3d65502dd8cb6758ddb5,0x7271c13a0847eecb082b5efc35987ff972b944d084b1...,bostrom1uxa2sxtv7npaj6863fez5ccv2qfwszf2w7wga5,osmo1uxa2sxtv7npaj6863fez5ccv2qfwszf29kft4p,math escape slow grab honey ginger cage canoe ...,bostrom1uxa2sxtv7npaj6863fez5ccv2qfwszf2w7wga5...,0x98f805ceef6fbc3e185beb0bc25e03070da3997aad9f...,{'pub_key': 'AlgnuX3+tvoZEEbXKTuGpW/mQbSNBmoxB...,eyJwdWJfa2V5IjoiQWxnbnVYMyt0dm9aRUViWEtUdUdwVy...
2,26000,john16517597492,QmWdMd8DxpakrEidFnKqFgZwSNcBuVYVYMzvKNE82C7ZQw,0xc834f631039eba3e7fede0b3bee5f7b3aabcdc1f,0xf3a8658346c3888bdc236ccdd07532996eebbd299fa5...,bostrom12rwwttemkn7w3xanpqtu05euzn2nsjzfkxdpe9,terra12rwwttemkn7w3xanpqtu05euzn2nsjzfn3rj9z,kangaroo sport welcome kangaroo digital retrea...,bostrom12rwwttemkn7w3xanpqtu05euzn2nsjzfkxdpe9...,0xc611b87310ee2d1a2cf66626afeb0c05cd9361db397f...,{'pub_key': 'AkSMmBe8/3Z/x3FQDHdA0HmPC6gSS/VqI...,eyJwdWJfa2V5IjoiQWtTTW1CZTgvM1oveDNGUURIZEEwSG...
3,31000,john16517597493,QmaEcQYeQ6TzsoFmF54MU6p3RmUv22zfzv5jNhdRPDcV3g,0x9a2fcf7df51f24a72631694ba64745cbc511d4d3,0x683ab3cc400219f889b0ff07a8157d8065f856df6019...,bostrom15xdumxh2xyx5trteh689pkjkf3vkc0eqj3wxl6,terra15xdumxh2xyx5trteh689pkjkf3vkc0eqhxq4ra,poverty glass bid buyer pass morning vanish ri...,bostrom15xdumxh2xyx5trteh689pkjkf3vkc0eqj3wxl6...,0x64b80d0ae3e755e5ff94ac37f3d04c6254a104a6c178...,{'pub_key': 'Aqn4lUPTAD2ZiH2ncmLQBvUhxlnaxOmj+...,eyJwdWJfa2V5IjoiQXFuNGxVUFRBRDJaaUgybmNtTFFCdl...
4,35000,john16517597494,QmUbL8BAoJokFQsZg8iycc6rGnAvjNhP5JMaDHjaTzm4bb,0xa5528df7d1b90814092ce474869d37cd60a5fdd3,0xdcfb6e6b6e394536eb037666d5f67a1b10274d620f6a...,bostrom1463yk59gkwksn574mrye5v26qrqrqvk7mpsumm,osmo1463yk59gkwksn574mrye5v26qrqrqvk7sfhlnw,inhale sugar segment ethics number blouse dune...,bostrom1463yk59gkwksn574mrye5v26qrqrqvk7mpsumm...,0x18201d2d41ebf13ed6ec661468db58e1f008945cc79e...,{'pub_key': 'Ap41abfiY1qCYZSA+XKU2KhiPW322p0jj...,eyJwdWJfa2V5IjoiQXA0MWFiZmlZMXFDWVpTQStYS1UyS2...


### Create Merkle Tree, Get Root and Proofs

In [3]:
root_source_list = [{'address': _item[0],
                     'amount': str(_item[1])} for _item in claims_df[['ethereum_address', 'amount']].values.tolist()]+ \
                    [{'address': _item[0],
                      'amount': str(_item[1])} for _item in claims_df[['cosmos_address', 'amount']].values.tolist()]
root_source_list.append({'address': '0xF2749114FeaAD68854E01C8eE762C7170532FdfD'.lower(), 'amount': '10000000'})
with open(ROOT_SOURCE_FILE_NAME, 'w') as outfile:
    outfile.write(str(root_source_list).replace("'", '"'))

In [5]:
NUMBER_OF_THREADS = 30
BASH_SIZE = 2000

tasks = set(
    (
        (ROOT_SOURCE_FILE_NAME,
         f'temp/proofs_{i}.json',
         i * BASH_SIZE,
         min(NUMBER_OF_PARTICIPANTS * 2 + 1, (i + 1) * BASH_SIZE + 1))
        for i in range(ceil(2 * NUMBER_OF_PARTICIPANTS/BASH_SIZE))
    )
)
if CALCULATE_PROOFS:
    print(f'Number of addresses: {2 * NUMBER_OF_PARTICIPANTS}\nNumber of threads: {NUMBER_OF_THREADS}\nBash size: {BASH_SIZE}\nNumber of tasks: {len(tasks)}')
    with Pool(processes=NUMBER_OF_THREADS) as pool:
        res = pool.starmap(get_proofs, tqdm(tasks, total=len(tasks)))
    assert res == [True] * len(res)

Number of addresses: 400000
Number of threads: 30
Bash size: 2000
Number of tasks: 200


  0%|          | 0/200 [00:00<?, ?it/s]

yarn run v1.22.18
$ ts-node index.ts --input root_testing_source_200000_addresses.json --output temp/proofs_61.json --start_index 122000 --end_index 124001
Merkle root: c70fea2abe34353390986cd948a8193b9540d29aeda030f39a1ff2852557c90d
Number of addresses in the Merkle tree: 2001
Done in 318.34s.

yarn run v1.22.18
$ ts-node index.ts --input root_testing_source_200000_addresses.json --output temp/proofs_101.json --start_index 202000 --end_index 204001
Merkle root: c70fea2abe34353390986cd948a8193b9540d29aeda030f39a1ff2852557c90d
Number of addresses in the Merkle tree: 2001
Done in 319.03s.

yarn run v1.22.18
$ ts-node index.ts --input root_testing_source_200000_addresses.json --output temp/proofs_97.json --start_index 194000 --end_index 196001
Merkle root: c70fea2abe34353390986cd948a8193b9540d29aeda030f39a1ff2852557c90d
Number of addresses in the Merkle tree: 2001
Done in 319.20s.

yarn run v1.22.18
$ ts-node index.ts --input root_testing_source_200000_addresses.json --output temp/proofs_

In [6]:
if CALCULATE_PROOFS:
    roots = []
    proofs_df = pd.DataFrame(columns=['address', 'amount', 'proof'])
    for task in tasks:
        with open(task[1], 'r') as proof_file:
            root_and_proof_json = json.load(proof_file)
        roots.append(root_and_proof_json['merkle_root'])
        proofs_df = pd.concat([proofs_df, pd.DataFrame(root_and_proof_json['proofs'])], ignore_index=True)
    assert roots == [roots[0]] * len(roots)
    root = roots[0]

    cosmos_proofs_df = proofs_df[~proofs_df.address.str.startswith('0x')][['address', 'proof']]
    ethereum_proofs_df = proofs_df[proofs_df.address.str.startswith('0x')][['address', 'proof']]

    claims_with_proofs_df = claims_df\
        .merge(
            ethereum_proofs_df.rename(columns={'address': 'ethereum_address', 'proof': 'ethereum_proof'}),
            how='left',
            on='ethereum_address')\
        .merge(
            cosmos_proofs_df.rename(columns={'address': 'cosmos_address', 'proof': 'cosmos_proof'}),
            how='left',
            on='cosmos_address')

    claims_with_proofs_df.to_csv(PARTICIPANTS_FILE_NAME)
else:
    claims_with_proofs_df = pd.read_csv(PARTICIPANTS_FILE_NAME)
    claims_with_proofs_df.loc[:, 'ethereum_proof'] = claims_with_proofs_df['ethereum_proof'].map(lambda x: x.replace('\'', '').replace('[', '').replace(']', '').split(', '))
    claims_with_proofs_df.loc[:, 'cosmos_proof'] = claims_with_proofs_df['cosmos_proof'].map(lambda x: x.replace('\'', '').replace('[', '').replace(']', '').split(', '))
    with open('temp/proofs_1.json', 'r') as proof_file:
        root_and_proof_json = json.load(proof_file)
    root = root_and_proof_json['merkle_root']
claims_with_proofs_df.head()

,amount,nickname,avatar,ethereum_address,ethereum_private_key,bostrom_address,cosmos_address,cosmos_seed,message,ethereum_message_signature,cosmos_message_signed_row,cosmos_message_signature,ethereum_proof,cosmos_proof
0,33000,john16517597490,Qme4FqA7kkwm4G14MEBXUGycmLN1j3CAaNgfYoysHZK5fF,0xb13349fc01611fb3d5eaa220ce20d7935503bfdb,0x87b3b8f2bb51d574cf3c97302f01bc71e0f77746d530...,bostrom16jeqp7f4s97g0rkq3wqdwwr34ntn2f503rj59z,terra16jeqp7f4s97g0rkq3wqdwwr34ntn2f5055u8e9,wheel saddle egg habit ribbon chase pilot focu...,bostrom16jeqp7f4s97g0rkq3wqdwwr34ntn2f503rj59z...,0x4feef418397ebfd0cae573e031bebe1f69d486c6ee50...,{'pub_key': 'AjX/LeG0AQo/SXxmETFE4msQBpGN1w028...,eyJwdWJfa2V5IjoiQWpYL0xlRzBBUW8vU1h4bUVURkU0bX...,[7cf6e91feee2215ed6f455ffe4b5f1f51f4e1d7e05349...,[337592eb65bf5bd9a5ca8618c1bf1ec640f84a9c4da12...
1,33000,john16517597490,Qme4FqA7kkwm4G14MEBXUGycmLN1j3CAaNgfYoysHZK5fF,0xb13349fc01611fb3d5eaa220ce20d7935503bfdb,0x87b3b8f2bb51d574cf3c97302f01bc71e0f77746d530...,bostrom16jeqp7f4s97g0rkq3wqdwwr34ntn2f503rj59z,terra16jeqp7f4s97g0rkq3wqdwwr34ntn2f5055u8e9,wheel saddle egg habit ribbon chase pilot focu...,bostrom16jeqp7f4s97g0rkq3wqdwwr34ntn2f503rj59z...,0x4feef418397ebfd0cae573e031bebe1f69d486c6ee50...,{'pub_key': 'AjX/LeG0AQo/SXxmETFE4msQBpGN1w028...,eyJwdWJfa2V5IjoiQWpYL0xlRzBBUW8vU1h4bUVURkU0bX...,[7cf6e91feee2215ed6f455ffe4b5f1f51f4e1d7e05349...,[337592eb65bf5bd9a5ca8618c1bf1ec640f84a9c4da12...
2,36000,john16517597491,QmPwSc85Jmiw4mTS4esQNnPRwHgiXDFSVqPX7Dquv67L61,0xd4401c790797baff16eb3d65502dd8cb6758ddb5,0x7271c13a0847eecb082b5efc35987ff972b944d084b1...,bostrom1uxa2sxtv7npaj6863fez5ccv2qfwszf2w7wga5,osmo1uxa2sxtv7npaj6863fez5ccv2qfwszf29kft4p,math escape slow grab honey ginger cage canoe ...,bostrom1uxa2sxtv7npaj6863fez5ccv2qfwszf2w7wga5...,0x98f805ceef6fbc3e185beb0bc25e03070da3997aad9f...,{'pub_key': 'AlgnuX3+tvoZEEbXKTuGpW/mQbSNBmoxB...,eyJwdWJfa2V5IjoiQWxnbnVYMyt0dm9aRUViWEtUdUdwVy...,[da1f10a4786db051f14e7036d366179ea44979e4e434b...,[ecce50b48369ad2087983352a8338d81fd15aaa78a3c3...
3,26000,john16517597492,QmWdMd8DxpakrEidFnKqFgZwSNcBuVYVYMzvKNE82C7ZQw,0xc834f631039eba3e7fede0b3bee5f7b3aabcdc1f,0xf3a8658346c3888bdc236ccdd07532996eebbd299fa5...,bostrom12rwwttemkn7w3xanpqtu05euzn2nsjzfkxdpe9,terra12rwwttemkn7w3xanpqtu05euzn2nsjzfn3rj9z,kangaroo sport welcome kangaroo digital retrea...,bostrom12rwwttemkn7w3xanpqtu05euzn2nsjzfkxdpe9...,0xc611b87310ee2d1a2cf66626afeb0c05cd9361db397f...,{'pub_key': 'AkSMmBe8/3Z/x3FQDHdA0HmPC6gSS/VqI...,eyJwdWJfa2V5IjoiQWtTTW1CZTgvM1oveDNGUURIZEEwSG...,[2aa552a7021fc58abe81a3ecb586a774280bc8fd5ec45...,[eebba466aa32657b543e235662293878a16080b4268d9...
4,31000,john16517597493,QmaEcQYeQ6TzsoFmF54MU6p3RmUv22zfzv5jNhdRPDcV3g,0x9a2fcf7df51f24a72631694ba64745cbc511d4d3,0x683ab3cc400219f889b0ff07a8157d8065f856df6019...,bostrom15xdumxh2xyx5trteh689pkjkf3vkc0eqj3wxl6,terra15xdumxh2xyx5trteh689pkjkf3vkc0eqhxq4ra,poverty glass bid buyer pass morning vanish ri...,bostrom15xdumxh2xyx5trteh689pkjkf3vkc0eqj3wxl6...,0x64b80d0ae3e755e5ff94ac37f3d04c6254a104a6c178...,{'pub_key': 'Aqn4lUPTAD2ZiH2ncmLQBvUhxlnaxOmj+...,eyJwdWJfa2V5IjoiQXFuNGxVUFRBRDJaaUgybmNtTFFCdl...,[5b762b4b88c7e19883546c11d83858cc7462925fdbb5f...,[7350ba83595c6019a8df9d828592fa842ed525e36e5c3...


In [7]:
for i in range(ceil(NUMBER_OF_PARTICIPANTS/10_000)):
    claims_with_proofs_df.loc[i*10_000:(i+1)*10_000,:].to_csv(f'temp/{i}_{PARTICIPANTS_FILE_NAME}')

## Instantiate Contracts
### Instantiate SUBGRAPH Contracts

In [8]:
if INIT_SUBGRAPH_CONTRACTS:
    name_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test name subgraph')
    avatar_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test avatar subgraph',
            from_address=WALLET_ADDRESS)
    proof_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test proof subgraph',
            from_address=WALLET_ADDRESS)
else:
    name_subgraph_contract_address = NAME_SUBGRAPH_CONTRACT_ADDRESS
    avatar_subgraph_contract_address = AVATAR_SUBGRAPH_CONTRACT_ADDRESS
    proof_subgraph_contract_address = PROOF_SUBGRAPH_CONTRACT_ADDRESS
print(f'Name subgraph contract address: {name_subgraph_contract_address}\n'
      f'Avatar subgraph contract address: {avatar_subgraph_contract_address}\n'
      f'Proof subgraph contract address: {proof_subgraph_contract_address}')

Name subgraph contract address: bostrom1qg5ega6dykkxc307y25pecuufrjkxkaggkkxh7nad0vhyhtuhw3sp98tza
Avatar subgraph contract address: bostrom17p9rzwnnfxcjp32un9ug7yhhzgtkhvl9jfksztgw5uh69wac2pgs6qwudf
Proof subgraph contract address: bostrom1unyuj8qnmygvzuex3dwmg9yzt9alhvyeat0uu0jedg2wj33efl5qpj0rgp


### Instantiate Passport Contract

In [9]:
if INIT_PASSPORT_CONTRACT:
    passport_contract_address = \
        instantiate_contract(
            init_query=f'''{{"name":"CPT", "minter":"{WALLET_ADDRESS}", "owner":"{WALLET_ADDRESS}", "symbol":"CPT", "avatar_subgraph": "{avatar_subgraph_contract_address}", "name_subgraph": "{name_subgraph_contract_address}", "proof_subgraph": "{proof_subgraph_contract_address}"}}''',
            contract_code_id=PASSPORT_CODE_ID,
            contract_label='test passport',
            from_address=WALLET_ADDRESS)
else:
    passport_contract_address = PASSPORT_CONTRACT_ADDRESS
print(f'Passport contract address: {passport_contract_address}')

Passport contract address: bostrom1hulx7cgvpfcvg83wk5h96sedqgn72n026w6nl47uht554xhvj9nsjxcwgf


### Set executor in the Subgraph Contracts

In [10]:
def set_executor_subgraph(subgraph_contract_address: str, new_executor_address: str, display_data: bool = False):
    return execute_contract_bash(execute_query=f'''{{"update_executer":{{"new_executer":"{new_executor_address}"}}}}''',
                                 contract_address=subgraph_contract_address,
                                 gas=600000,
                                 display_data=display_data)

if INIT_PASSPORT_CONTRACT or INIT_SUBGRAPH_CONTRACTS:
    set_executor_subgraph(subgraph_contract_address=name_subgraph_contract_address, new_executor_address=passport_contract_address)
    set_executor_subgraph(subgraph_contract_address=avatar_subgraph_contract_address, new_executor_address=passport_contract_address)
    set_executor_subgraph(subgraph_contract_address=proof_subgraph_contract_address, new_executor_address=passport_contract_address)

### Instantiate Treasury Contract

In [11]:
if INIT_TREASURY_CONTRACT:
    treasury_contract_address = \
        instantiate_contract(
            init_query=f'''{{"admins": ["{WALLET_ADDRESS}"], "mutable": true}}''',
            contract_code_id=TREASURY_CODE_ID,
            amount=INITIAL_BALANCE,
            contract_label='test treasury',
            from_address=WALLET_ADDRESS,
            display_data=False)
else:
    treasury_contract_address = TREASURY_CONTRACT_ADDRESS
print(f'Treasury contract address: {treasury_contract_address}')

Treasury contract address: bostrom1nwp0ynjv84wxysf2f5ctvysl6dpm8ngm70hss6jeqt8q7e7u345s7fdeev


### Instantiate Gift Contract

In [12]:
if INIT_GIFT_CONTRACT:
    gift_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "passport":"{passport_contract_address}", "treasury":"{treasury_contract_address}", "allowed_native":"boot", "initial_balance":"{INITIAL_BALANCE}", "coefficient_up":"{COEF_UP}", "coefficient_down":"{COEF_DOWN}", "coefficient":"{COEF_UP}", "target_claim":"{TARGET_CLAIM}"}}''',
            contract_code_id=GIFT_CODE_ID,
            contract_label='test gift',
            from_address=WALLET_ADDRESS,
            display_data=False)
else:
    gift_contract_address = GIFT_CONTRACT_ADDRESS
print(f'Gift contract address: {gift_contract_address}')

Gift contract address: bostrom18a0pvw326fydfdat5tzyf4t8lhz0v6fyfaujpeg07fwqkygcxejs7l7gtg


### Initiate Class of Output Parsing

In [13]:
contract_utils = ContractUtils(ipfs_client=ipfs_client,
                               address_dict= {
                                   gift_contract_address: 'Gift Contract',
                                   passport_contract_address: 'Passport Contract',
                                   treasury_contract_address: 'Treasury Contract',
                                   WALLET_ADDRESS: 'Passport Owner Address',
                                   name_subgraph_contract_address: 'Name Subgraph Contract',
                                   avatar_subgraph_contract_address: 'Avatar Subgraph Contract',
                                   proof_subgraph_contract_address: 'Proof Subgraph Contract'})

### Set Allowance in Treasury contract

In [14]:
contract_utils.parse_contract_execution_json(
    contract_utils.send_coins(
        from_seed=WALLET_SEED,
        to_addresses=[treasury_contract_address],
        amounts=[INITIAL_BALANCE],
        gas=min(100_000 + 25_000, int(20e6)),
        display_data=DISPLAY_TX_EXECUTION))


Events

coin received
	receiver: Treasury Contract
	amount: 55000000000boot

coin spent
	spender: Passport Owner Address
	amount: 55000000000boot

message from bank bostrom1mxdtr8lruutugqtxgpw2sf2tl2mhzlq5fd2du0 /cosmos.bank.v1beta1.MsgMultiSend

transfer
	recipient: Treasury Contract
	amount: 55000000000boot
Gas used: 73,258
Tx hash: 86B2B41DBDF0F0F7E3D8F73F92F443B5BCF550BC72DD7E7394066459427FD553


In [15]:
def set_allowance(amount: str = INITIAL_BALANCE, display_data: bool = False):
    return contract_utils.execute_contract(
        execute_msg={"increase_allowance": {"spender": gift_contract_address, "amount": {"amount": amount, "denom": 'boot'}}},
        contract_address=treasury_contract_address,
        mnemonic=WALLET_SEED,
        gas=600000,
        display_data=display_data)

contract_utils.parse_contract_execution_json(
    set_allowance(display_data=False))


Events

execute
	execute contract: Treasury Contract

message from Passport Owner Address wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Treasury Contract
	action: increase_allowance
	owner: Passport Owner Address
	spender: Gift Contract
	denomination: boot
	amount: 55,000,000,000
Gas used: 135,324
Tx hash: 3A738CC9C42A96E5DFC59919ACA7FD50B8F7027B7C016CD3C49390F51A7576E4


### Register Merkle Root

In [16]:
root_register_output = contract_utils.execute_contract(
    execute_msg={"register_merkle_root": {"merkle_root": root}},
    mnemonic=WALLET_SEED,
    contract_address=gift_contract_address)
contract_utils.parse_contract_execution_json(root_register_output)


Events

execute
	execute contract: Gift Contract

message from Passport Owner Address wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Gift Contract
	action: register_merkle_root
	merkle_root: c70fea2abe34353390986cd948a8193b9540d29aeda030f39a1ff2852557c90d
Gas used: 131,205
Tx hash: AEA967908317DCFD2756C39E6389D672FF0019F29FB0A26F45001CA77154121C


#### Get Merkle Root form the Gift Contract

In [17]:
print(f'Gift contract {gift_contract_address}')
print(query_contract(query='''{"merkle_root": {}}''',
                     contract_address=gift_contract_address))

Gift contract bostrom18a0pvw326fydfdat5tzyf4t8lhz0v6fyfaujpeg07fwqkygcxejs7l7gtg
{'data': {'merkle_root': 'c70fea2abe34353390986cd948a8193b9540d29aeda030f39a1ff2852557c90d'}}


### Send coins to new addresses

In [18]:
NUMBER_ADDRESSES_IN_SENDING_CHUNK = 750

bostrom_addresses = claims_with_proofs_df.bostrom_address.to_list()
bostrom_addresses_chunks = [bostrom_addresses[i: i+ NUMBER_ADDRESSES_IN_SENDING_CHUNK] for i in range(0, len(bostrom_addresses), NUMBER_ADDRESSES_IN_SENDING_CHUNK)]

for bostrom_addresses_item in tqdm(bostrom_addresses_chunks):
    try:
        send_output = contract_utils.send_coins(
                from_seed=WALLET_SEED,
                to_addresses=bostrom_addresses_item,
                amounts=[1] * len(bostrom_addresses_item),
                gas=min(150_000 + 30_000 * len(bostrom_addresses_item), int(24e6)),
                display_data=DISPLAY_TX_EXECUTION)
    except Exception as e:
        print(f'Error: {e}')

  0%|          | 0/268 [00:00<?, ?it/s]

In [19]:
contract_utils.parse_contract_execution_json(contract_execution_json=send_output)


Events

coin received
	receiver: bostrom10xy8tvszahr6nmprggqcz3sgzfa2xgajxscp6q
	amount: 1boot
	receiver: bostrom1k4w7m98rh9p74nq83ngqzaj2wrls9wsf8tpwus
	amount: 1boot
	receiver: bostrom1ukchvl0g8ts8pj7x7smlujetyjn8fz6cet4yxg
	amount: 1boot
	receiver: bostrom14ccsryca9ddf5gd7t3rky4ru4xrxu50lf6lxs5
	amount: 1boot
	receiver: bostrom1cdwtuv9z2s3mctjhy9plgwjwhvg2r9dnjdv5f6
	amount: 1boot
	receiver: bostrom18847espeh28xl7g5sxn4508s7h4lgjrgwl6ve7
	amount: 1boot
	receiver: bostrom15ttpn4cp85akn2xd54xnhv0rw56qry83hhc55f
	amount: 1boot
	receiver: bostrom1778cm5cmq4m6f3xhjen56fprpyjwfsvlcz05la
	amount: 1boot
	receiver: bostrom1rfz9rrxelqt4x3v8xsuj98g7p79u5aua8e7nee
	amount: 1boot
	receiver: bostrom1uqmr30yvmwq7ptrnl6ek2znjjd9jh3kvvnt7gv
	amount: 1boot
	receiver: bostrom1zuygpx37d2narzu8a5gk8fgwd4h50m9qqpdejc
	amount: 1boot
	receiver: bostrom125cdfp4jr0u78fjkgy84tlvnemmavsn7rz892q
	amount: 1boot
	receiver: bostrom1xn269mms0endv0kx3chrlrjyhxtyq9n28tgg2v
	amount: 1boot
	receiver: bostrom1h2aevm6um

### Create Passports, Proof Addresses and Claim Gift

In [20]:
def execute_participation(row_index: int) -> bool:
    _output, _error = execute_bash(bash_command=f'{PYTHON_PATH} create_passport_and_claim_job.py temp/{row_index//10_000}_{PARTICIPANTS_FILE_NAME} {row_index} {gift_contract_address}')
    if _output:
        return _output
    else:
        return _error

NUMBER_OF_THREADS = 100

tasks = list(range(NUMBER_OF_ACTIVATED_PARTICIPANTS))
print(f'Number of tasks: {len(tasks)}')
print(f'Number of threads: {NUMBER_OF_THREADS}')


with Pool(processes=NUMBER_OF_THREADS) as pool:
    res_participation = list(tqdm(pool.imap(execute_participation, tasks), total=len(tasks)))

Number of tasks: 100000
Number of threads: 100


  0%|          | 0/100000 [00:00<?, ?it/s]

In [21]:
with open('temp/contract_participation_execution_log_1.txt', 'r') as f:
    data = json.load(f)

In [22]:
contract_utils.parse_contract_execution_json(data['create'], row=claims_with_proofs_df.loc[1,:])


Events

cyberlinks
	Passport Owner Address -> Nickname
	Nickname -> Passport Owner Address
	neuron: Name Subgraph Contract

	Nickname -> Avatar
	Avatar -> Nickname
	neuron: Avatar Subgraph Contract


execute
	execute contract: Passport Contract
	execute contract: Name Subgraph Contract
	execute contract: Avatar Subgraph Contract

message from bostrom16jeqp7f4s97g0rkq3wqdwwr34ntn2f503rj59z wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Passport Contract
	action: mint
	minter: Passport Contract
	owner: bostrom16jeqp7f4s97g0rkq3wqdwwr34ntn2f503rj59z
	token_id: 17
Gas used: 440,955
Tx hash: 62328D0646F7ED41BA9F6EFA2C816B9DD28EA97FFE5EEFC57290F3DA416AE6F6


In [23]:
contract_utils.parse_contract_execution_json(data['proof_ethereum'], row=claims_with_proofs_df.loc[1,:])


Events

cyberlinks
	Nickname -> Ethereum Address
	Ethereum Address -> Nickname
	neuron: Proof Subgraph Contract


execute
	execute contract: Passport Contract
	execute contract: Proof Subgraph Contract

message from bostrom16jeqp7f4s97g0rkq3wqdwwr34ntn2f503rj59z wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Passport Contract
	action: proof_address
	nickname: john16517597490
	address: 0xb13349fc01611fb3d5eaa220ce20d7935503bfdb
Gas used: 281,816
Tx hash: 07B96B6A84ABEE4F26F9F2B9A2223BF589FA22F6E5E2DAB2A8EE514B3744ABB4


In [24]:
contract_utils.parse_contract_execution_json(data['proof_cosmos'], row=claims_with_proofs_df.loc[1,:])


Events

cyberlinks
	Nickname -> Cosmos Address
	Cosmos Address -> Nickname
	neuron: Proof Subgraph Contract


execute
	execute contract: Passport Contract
	execute contract: Proof Subgraph Contract

message from bostrom16jeqp7f4s97g0rkq3wqdwwr34ntn2f503rj59z wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Passport Contract
	action: proof_address
	nickname: john16517597490
	address: terra16jeqp7f4s97g0rkq3wqdwwr34ntn2f5055u8e9
Gas used: 285,198
Tx hash: 6F48F5C93D32EF5C3F0CA1A494D34198BF55E9CA6BE5957EAC32071325591BFA


In [25]:
contract_utils.parse_contract_execution_json(data['claim_cosmos'], row=claims_with_proofs_df.loc[1,:])


Events
account sequence mismatch, expected 5, got 4: incorrect account sequence
Gas used: 67,733
Tx hash: 6EB9BA76D0C7C235D2D249C8347E6603449ABE65E657C2FA1506EA1EFBE06F4A


### Release Gift

In [26]:
def execute_release(row_index: int) -> bool:
    _output, _error = execute_bash(bash_command=f'{PYTHON_PATH} create_passport_and_claim_job.py temp/{row_index//10_000}_{PARTICIPANTS_FILE_NAME} {row_index} {gift_contract_address} True')
    if _output:
        return _output
    else:
        return _error


NUMBER_OF_THREADS = 100

tasks = list(range(NUMBER_OF_ACTIVATED_PARTICIPANTS))
print(f'Number of tasks: {len(tasks)}')
print(f'Number of threads: {NUMBER_OF_THREADS}')


with Pool(processes=NUMBER_OF_THREADS) as pool:
    res_release = list(tqdm(pool.imap(execute_release, tasks), total=len(tasks)))

Number of tasks: 100000
Number of threads: 100


  0%|          | 0/100000 [00:00<?, ?it/s]

In [27]:
with open('temp/contract_release_execution_log_1.txt', 'r') as f:
    release_data = json.load(f)

In [28]:
contract_utils.parse_contract_execution_json(release_data['release_ethereum'], row=claims_with_proofs_df.loc[1,:])


Events

coin received
	receiver: bostrom16jeqp7f4s97g0rkq3wqdwwr34ntn2f503rj59z
	amount: 32891boot

coin spent
	spender: Treasury Contract
	amount: 32891boot

execute
	execute contract: Gift Contract
	execute contract: Treasury Contract

message from bostrom16jeqp7f4s97g0rkq3wqdwwr34ntn2f503rj59z wasm /cosmwasm.wasm.v1.MsgExecuteContract

transfer
	recipient: bostrom16jeqp7f4s97g0rkq3wqdwwr34ntn2f503rj59z
	sender: Treasury Contract
	amount: 32891boot

wasm
	_contract_address: Gift Contract
	action: release
	address: bostrom16jeqp7f4s97g0rkq3wqdwwr34ntn2f503rj59z
	gift_address: 0xb13349fc01611fb3d5eaa220ce20d7935503bfdb
	stage: 9
	amount: 32,891
	_contract_address: Treasury Contract
	action: execute
	owner: Gift Contract
Gas used: 230,190
Tx hash: 355BBB42ADA98FDA99AFF40DB58DA406E8590AFC915A1FF3708251A4BE125C23


In [29]:
contract_utils.parse_contract_execution_json(release_data['release_cosmos'], row=claims_with_proofs_df.loc[1,:])


Events

coin received
	receiver: bostrom16jeqp7f4s97g0rkq3wqdwwr34ntn2f503rj59z
	amount: 32871boot

coin spent
	spender: Treasury Contract
	amount: 32871boot

execute
	execute contract: Gift Contract
	execute contract: Treasury Contract

message from bostrom16jeqp7f4s97g0rkq3wqdwwr34ntn2f503rj59z wasm /cosmwasm.wasm.v1.MsgExecuteContract

transfer
	recipient: bostrom16jeqp7f4s97g0rkq3wqdwwr34ntn2f503rj59z
	sender: Treasury Contract
	amount: 32871boot

wasm
	_contract_address: Gift Contract
	action: release
	address: bostrom16jeqp7f4s97g0rkq3wqdwwr34ntn2f503rj59z
	gift_address: terra16jeqp7f4s97g0rkq3wqdwwr34ntn2f5055u8e9
	stage: 9
	amount: 32,871
	_contract_address: Treasury Contract
	action: execute
	owner: Gift Contract
Gas used: 230,374
Tx hash: B188F92030CBACDBC26E230438BFC22A92E1CB2260C47B9345E9CFCAF0B8F9F7


## Passport NFT testing
### Transfer Passport

In [30]:
def get_passport_id(bostrom_address: str) -> str:
    return query_contract(query=f'''{{"tokens": {{"owner": "{bostrom_address}"}}}}''',
                          contract_address=passport_contract_address)['data']['tokens'][0]

In [31]:
for index, row in tqdm(claims_with_proofs_df[:4].iterrows()):
    if index == 1:
        transfer_passport_json = contract_utils.transfer_passport(
            row,
            token_id=get_passport_id(row['bostrom_address']),
            to_address=WALLET_ADDRESS,
            display_data=DISPLAY_TX_EXECUTION)
        contract_utils.parse_contract_execution_json(
            transfer_passport_json,
            row=row)
    else:
        pass

0it [00:00, ?it/s]


Events

cyberlinks
	QmPcFZbajorUYkkdeowYUD1zJRAWA13N4Koy4rLeZ4cDji -> Nickname
	Nickname -> QmPcFZbajorUYkkdeowYUD1zJRAWA13N4Koy4rLeZ4cDji
	neuron: Name Subgraph Contract


execute
	execute contract: Passport Contract
	execute contract: Name Subgraph Contract
	execute contract: Avatar Subgraph Contract

message from bostrom16jeqp7f4s97g0rkq3wqdwwr34ntn2f503rj59z wasm /cosmwasm.wasm.v1.MsgExecuteContract

reply
	reply contract: Avatar Subgraph Contract

wasm
	_contract_address: Passport Contract
	action: transfer_nft
	sender: bostrom16jeqp7f4s97g0rkq3wqdwwr34ntn2f503rj59z
	recipient: Passport Owner Address
	token_id: 17
Gas used: 387,001
Tx hash: D7FA27D76C9FB6DF503623AD064BEC9E1F79B83036CB56053F855922E12D36B8


### Burn

In [32]:
for index, row in tqdm(claims_with_proofs_df[2:6].iterrows()):
    if index == 2:
        burn_passport_json = contract_utils.burn_passport(row, token_id=get_passport_id(row['bostrom_address']), display_data=DISPLAY_TX_EXECUTION)
        contract_utils.parse_contract_execution_json(burn_passport_json, row=row)
    else:
        pass

0it [00:00, ?it/s]


Events

cyberlinks
	cyberhole -> Nickname
	Nickname -> cyberhole
	neuron: Name Subgraph Contract

	cyberhole -> Avatar
	Avatar -> cyberhole
	neuron: Avatar Subgraph Contract


execute
	execute contract: Passport Contract
	execute contract: Name Subgraph Contract
	execute contract: Avatar Subgraph Contract

message from bostrom1uxa2sxtv7npaj6863fez5ccv2qfwszf2w7wga5 wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Passport Contract
	action: burn
	sender: bostrom1uxa2sxtv7npaj6863fez5ccv2qfwszf2w7wga5
	token_id: 101
Gas used: 392,366
Tx hash: FB209489737B8CDBB7E642C6ACC55EE894053A2A799F3C5C247050E73267AEEA


### Add tests for Passport contract
#### create_passport +
#### update_name
#### update_avatar
#### proof_address +
#### remove_address
#### set_minter
#### set_owner
#### set_active
#### set_subgraphs
#### transfer_nft +
#### send_nft
#### mint
#### burn +
#### approve
#### approve_all
#### revoke
#### revoke_all
#### Expirations
##### at_height
##### at_time
##### never

### Add tests for Gift contract
#### update_owner
#### update_passport_addr
#### update_target
#### register_merkle_root +
#### claim +
#### release +

### Add tests for subgraph contract
#### update_owner
#### update_executer +
